In [1]:
import os
import pandas as pd
import numpy as np
import torch
import re
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, Trainer, TrainingArguments,EsmForSequenceClassification, AdamW,EsmTokenizer
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

In [2]:
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [ ]:
print("cuda is on" if torch.cuda.is_available() else "cpu")

cuda is on 
God SpeeD 🏎️🏎️🏎️🏎️


In [4]:
train_data = pd.read_csv("/home/ubuntu/work/data/new/mini/balanced/train_data4.csv")
test_data = pd.read_csv("/home/ubuntu/work/data/new/mini/balanced/test_data.csv")

In [5]:
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D", do_lower_case=False)
model = EsmForSequenceClassification.from_pretrained("/home/ubuntu/work/codes/multi fine tuning4/tune4/best_model4")

In [6]:
class processor(Dataset):
    def __init__(self, df, tokenizer_name="facebook/esm2_t6_8M_UR50D", max_len=24):
        self.tokenizer = EsmTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        self.max_len = max_len
        self.seqs, self.labels = self.get_seqs_labels(df)

    def get_seqs_labels(self, df):
        seqs = list(df['seq'])
        labels = list(df['type'].astype(int))
        return seqs, labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        seq = " ".join("".join(self.seqs[idx].split()))
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_len)
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

In [7]:
def model_init():
    return  EsmForSequenceClassification.from_pretrained("/home/ubuntu/work/codes/multi fine tuning4/tune4/best_model4")


In [8]:
trainingdata  = processor(train_data)
evaldata = processor(test_data)

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Predicted classes
    probs = pred.predictions[:, 1]       # Probabilities for positive class

    # Calculate basic metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    
    # Calculate confusion matrix for specificity
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    specificity = tn / (tn + fp)
    
    # Calculate AUC-ROC score
    auc = roc_auc_score(labels, probs)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'auc_roc': auc
    }

In [ ]:

# Define the hyperparameters for training (from hyperparameter search)
training_args = TrainingArguments(
    output_dir='./results',                    # Output directory to save the model
    num_train_epochs=5,                        # Number of training epochs
    learning_rate=  2.1430509048903236e-05,                        # Learning rate for the optimizer
    per_device_train_batch_size=16,            # Batch size per device during training
    warmup_steps=500,                          # Number of warmup steps for learning rate scheduler
    weight_decay= 0.13684509966496455,                         # Strength of weight decay
    logging_dir='./logs',                      # Directory for storing logs
    logging_steps=100,                         # Frequency of logging
    evaluation_strategy="epoch",               # Evaluate the model every epoch
                   # Save the model after every epoch
    gradient_accumulation_steps=8,             # Number of steps to accumulate gradients
    fp16=True,                                 # Enable mixed precision training (if possible)
                        # Random seed for reproducibility
)

# Initialize Trainer with model and dataset
trainer = Trainer(
    model=model,                              # The model you want to fine-tune
    args=training_args,                       # The training arguments defined above
    train_dataset=trainingdata,              # Your training dataset
    eval_dataset=evaldata,                # Your evaluation dataset (optional, can be omitted)
    compute_metrics=compute_metrics           # Optional: function to compute evaluation metrics
)

# Fine-tune the model
trainer.train()

# Evaluate the model after training
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


/home/ubuntu/anaconda3/envs/sathdeep/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.46441447734832764, 'eval_accuracy': 0.85, 'eval_f1': 0.8421052631578947, 'eval_precision': 0.8888888888888888, 'eval_recall': 0.8, 'eval_specificity': 0.9, 'eval_auc_roc': 0.9036111111111111, 'eval_runtime': 0.0799, 'eval_samples_per_second': 1500.963, 'eval_steps_per_second': 187.62, 'epoch': 0.73}


  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 0.4668828248977661, 'eval_accuracy': 0.8416666666666667, 'eval_f1': 0.831858407079646, 'eval_precision': 0.8867924528301887, 'eval_recall': 0.7833333333333333, 'eval_specificity': 0.9, 'eval_auc_roc': 0.9034722222222221, 'eval_runtime': 0.0506, 'eval_samples_per_second': 2370.567, 'eval_steps_per_second': 296.321, 'epoch': 2.91}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.4693514406681061, 'eval_accuracy': 0.8333333333333334, 'eval_f1': 0.8214285714285714, 'eval_precision': 0.8846153846153846, 'eval_recall': 0.7666666666666667, 'eval_specificity': 0.9, 'eval_auc_roc': 0.9036111111111111, 'eval_runtime': 0.0513, 'eval_samples_per_second': 2340.8, 'eval_steps_per_second': 292.6, 'epoch': 3.64}
{'train_runtime': 1.3578, 'train_samples_per_second': 1259.371, 'train_steps_per_second': 7.365, 'train_loss': 0.2322521686553955, 'epoch': 3.64}


  0%|          | 0/15 [00:00<?, ?it/s]

Evaluation results:
eval_loss: 0.4693514406681061
eval_accuracy: 0.8333333333333334
eval_f1: 0.8214285714285714
eval_precision: 0.8846153846153846
eval_recall: 0.7666666666666667
eval_specificity: 0.9
eval_auc_roc: 0.9036111111111111
eval_runtime: 0.0645
eval_samples_per_second: 1859.026
eval_steps_per_second: 232.378
epoch: 3.6363636363636362


In [11]:
model_save_path = "./best_model5"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel saved at {model_save_path}")


Model saved at ./best_model5
